### Activamos multi-threading en VSCode con JupyterNotebook

+ [https://docs.julialang.org/en/v1/manual/multi-threading/](https://docs.julialang.org/en/v1/manual/multi-threading/)
+ [https://enccs.github.io/Julia-for-HPC/multithreading/](https://enccs.github.io/Julia-for-HPC/multithreading/)
+ [https://stackoverflow.com/questions/71596187/changing-threads-in-settings-json-of-vs-code-for-julias-jupyter-notebooks](https://stackoverflow.com/questions/71596187/changing-threads-in-settings-json-of-vs-code-for-julias-jupyter-notebooks)

In [5]:
println("Number of threads = ", Threads.nthreads());

Number of threads = 4


# Jupiter Notebook para testear código
Ahora testeamos el código agregandole el factor de acoplamiento y resolviendo el problema 1D (únicamente)

In [1]:
include("../modules/module_schrodinger_equation_01.jl"); # módulo para construir grilla (1D)

  Activating project at `~/github_repositories/my_repositories/pde2022/final_project/gridap_makie`


VARIABLES GLOBALES:
m=1.0000 (mass)
ω=1.0000 (frecuency)
ħ=1.0000 (Planck constant)
γ=0.1000 (coupling)


x₁=0.0000 x₂=2.0000 y₁=0.0000 y₂=2.0000 (QHO origin position)


In [2]:
grid_type="simple_line";                            # tipo de grilla
Δxₕ=1.0/200.0;                                      # tamaño del elento 1D

dom_1D=(-25.0,25.0);

par_1D=(path_models,grid_type*"_01",dom_1D,Δxₕ);    # (path,name,dom,MeshSize)

model_1D=make_model(grid_type,par_1D);              # creamos modelo
write_data=false
if write_data
    writevtk(model_1D,path_images*"model_1D");      # escribimos el modelo para visualizar con Visit
end

# condiciones de contorno
dirichlet_tags_1D=["left_point","right_point"];
dirichlet_values_1D=[(0.0+im*0.0),(0.0+im*0.0)];

Ω_1D,dΩ_1D,Γ_1D,dΓ_1D=measures(model_1D,3,dirichlet_tags_1D);

reffe_1D=reference_FEspaces(lagrangian,Float64,2);

Vₕ_1D=TestFESpace(model_1D,reffe_1D;vector_type=Vector{ComplexF64},conformity=:H1,dirichlet_tags=dirichlet_tags_1D);
Uₕ_1D=TrialFESpace(Vₕ_1D,dirichlet_values_1D);

aₕ_1D,bₕ_1D=bilineal_forms(pₕ,qₕ,rₕ,dΩ_1D);

nevₕ=20 # number of eigenvalues asked to evaluate.
probₕ_1D=EigenProblem(aₕ_1D,bₕ_1D,Uₕ_1D,Vₕ_1D;nev=nevₕ,tol=10^(-6),maxiter=100,explicittransform=:none,sigma=-1.0)
ϵₕ_1D,ϕₕ_1D=solve(probₕ_1D);

Info    : Meshing 1D...
Info    : Meshing curve 1 (Line)
Info    : Done meshing 1D (Wall 0.00636014s, CPU 0.005754s)
Info    : 10001 nodes 10002 elements
Info    : Writing '../models/simple_line_01.msh'...
Info    : Done writing '../models/simple_line_01.msh'
Info    : Reading '../models/simple_line_01.msh'...
Info    : 3 entities
Info    : 10001 nodes
Info    : 10002 elements
Info    : Done reading '../models/simple_line_01.msh'


In [3]:
# Escribimos autovectores para visualizar con Visit
write_data=false;
if write_data
    for i in 1:nevₕ
        ϕₕi_1D = ϕₕ_1D[i]; # autovector asociado al autovalor 6
        writevtk(Ω_1D,path_images*"eigenprob_1D_01_num$(i)",cellfields=["phi_real" => real(ϕₕi_1D), "phi_imag" => real(ϕₕi_1D)]);
    end
end

> useful links to embeded mp4 inside Markdown Julia
 + [https://developer.mozilla.org/en-US/docs/Web/HTML/Element/video](https://developer.mozilla.org/en-US/docs/Web/HTML/Element/video)
 + [https://www.freeconvert.com/mpeg-to-mp4](https://www.freeconvert.com/mpeg-to-mp4)

### Ahora queremos resolver la dinámica del problema, es decir, la ecuación de evolución (ecuación de Schrödinger dependiente del tiempo)

### Defenimos el paso temporal

In [30]:
t₁=0.00001;     # final time for test
num_steps=5000; # number of time steps   
n_int=250;      # Intervals where data is saved
Δt=t₁/(num_steps-1) # time step

2.0004000800160032e-9

+ El factor $\frac {\Delta t}{\Delta x^2}$ nos permite corroborar que, si es pequeño $( \lt 1)$, el método de Euler usual funcionaría.
+ El factor $\frac {\Delta t}{\Delta x}$ si es aproximadamente igual a 1, y no usamos métodos implícitos, el método explícito sería intestable.
+ El factor $\left ( \frac {\Delta t}{\Delta x^2} \right)^3$ nos dice cómo escala el error del método implícito.

In [31]:
@printf("Δt/Δx²=%.6f,Δt/Δx=%.6f,Δt³/Δx⁶=%.6f",Δt/(Δxₕ^2),(Δt/Δxₕ),(Δt/(Δxₕ^2))^3)

Δt/Δx²=0.000080,Δt/Δx=0.000000,Δt³/Δx⁶=0.000000

### Creamos espacios de prueba y de solución

In [32]:
# espacio de pruebas y de soluciones para ψ₁
V₁H_1D=TestFESpace(model_1D,reffe_1D;vector_type=Vector{ComplexF64},conformity=:H1,dirichlet_tags=dirichlet_tags_1D);
U₁H_1D=TrialFESpace(V₁H_1D,dirichlet_values_1D);
# espacio de pruebas y de soluciones para ψ₂
V₂H_1D=TestFESpace(model_1D,reffe_1D;vector_type=Vector{ComplexF64},conformity=:H1,dirichlet_tags=dirichlet_tags_1D);
U₂H_1D=TrialFESpace(V₂H_1D,dirichlet_values_1D);
# creamos un campo de espacios
UH_1D = MultiFieldFESpace([U₁H_1D, U₂H_1D]);
VH_1D = MultiFieldFESpace([V₁H_1D, V₂H_1D]);

### Construimos la condición inicial

Para eso utilizamos la función de interpolación para crear objetos en un dominio específico

In [33]:
u₀ₕ_1D=interpolate_everywhere(ϕₕ_1D[1],Uₕ_1D);
interp_u₀ₕ_1D=Interpolable(u₀ₕ_1D);

In [34]:
u₁0H_1D_v1(x)=0.0;
u₂0H_1D_v1=interpolate_everywhere(interp_u₀ₕ_1D,U₂H_1D);

# u₂0H_1D_v2(x) = 0.0
# u₁0H_1D_v2=interpolate_everywhere(interp_u₀ₕ_1D,U₁H_1D);

SingleFieldFEFunction():
 num_cells: 10000
 DomainStyle: ReferenceDomain()
 Triangulation: BodyFittedTriangulation()
 Triangulation id: 13108042851827315708

In [35]:
u₀H_1D=interpolate_everywhere([u₁0H_1D_v1,u₂0H_1D_v1],UH_1D);
# u₀H_1D=interpolate_everywhere([u₁0H_1D_v2,u₂0H_1D_v2],UH_1D);

MultiFieldFEFunction():
 num_cells: 10000
 DomainStyle: ReferenceDomain()
 Triangulation: BodyFittedTriangulation()
 Triangulation id: 13108042851827315708

In [36]:
write_data=false
if write_data
    writevtk(Ω_2D,path_images*"evol_ini_dat_$(lpad(0,3,'0'))", cellfields=["u₁0H_2D_re"=>real(u₀H_2D[1]),"u₁0H_2D_im"=>imag(u₀H_2D[1])])
end

@printf("norm L₂u₁=%.6f,norm L₂u₂=%.6f",norm_L2(u₀H_1D[1],dΩ_1D),norm_L2(u₀H_1D[2],dΩ_1D))

norm L₂u₁=0.000000,norm L₂u₂=1.000000

In [37]:
assem_H=SparseMatrixAssembler(UH_1D,VH_1D);       # ensamblamos una matriz sparse con U y V
dv_H=get_fe_basis(VH_1D);                         # escribimos v en la base de FE de V            
du_H=get_trial_fe_basis(UH_1D);                   # escribimos u en la base de FE de U

Definimos forma bilinial a(u,v) y creamos matriz A

In [38]:
a_H_1D=a_bilineal_forms_2D(α₁,α₂,Δt,dΩ_1D);
mat_contribs_H=a_H_1D(du_H,dv_H);                       # creamos una matriz de bases (contribuciones)
data_H=collect_cell_matrix(UH_1D,VH_1D,mat_contribs_H); # recolectamos una matriz (datos)
A_H=assemble_matrix(assem_H,data_H);                    # creamos matriz A con contribuciones y datos

Realizamos la evolución, deiniendo en cada paso el funcional b(v) y haciendo uso del operador backslash para diagonalizar

In [39]:
t=0;  # inicializamos variable tiempo
write_data=true
if write_data
  writevtk(Ω_1D,path_images*"evol_1D_02_$(lpad(0,3,'0'))",
  cellfields=["u₁H_1D_re" => real(u₀H_1D[1]), "u₁H_1D_im" => imag(u₀H_1D[1]),"u₂H_1D_re" => real(u₀H_1D[2]), "u₂H_1D_im" => imag(u₀H_1D[2])])
  @printf("t=%.6f,norm L₂u₁=%.6f,norm L₂u₂=%.6f\n",t,norm_L2(u₀H_1D[1],dΩ_1D),norm_L2(u₀H_1D[2],dΩ_1D))
end

for istep in 1:num_steps
  # println("\n+++ Solving in step $istep of $num_steps +++\n")
  b_H_1D=b_bilineal_form_2D(α₁,α₂,u₀H_1D[1],u₀H_1D[2],Δt,dΩ_1D)

  # descomentar si queremos resolver de forma usual
  # vec_contribs_H=b_H_1D(dv_H[1],dv_H[2])
  vec_contribs_H=b_H_1D(dv_H)
  data_H=collect_cell_vector(VH_1D,vec_contribs_H)
  b_H=assemble_vector(assem_H,data_H)
  x_H = A_H \ b_H
  uH_1D = FEFunction(UH_1D,x_H)

  # descomentar si queremos resolver con solver (NO FUNCIONA)
  # op = Affineassem_HFEOperator(a_H_1D,b_H_1D,UH_1D,VH_1D)
  # ls=LUSolver();solver=LinearFESolver(ls)
  # uH_1D = solve(solver,op)

  u₀H_1D = uH_1D

  t = Δt*istep;

  if ((istep - 1) % n_int == 0) && (istep > 1) && (write_data)
    writevtk(Ω_1D,path_images*"evol_1D_02_$(lpad(istep,3,'0'))",
    cellfields=[ "u₁H_1D_re" => real(u₀H_1D[1]), "u₁H_1D_im" => imag(u₀H_1D[1]),"u₂H_1D_re" => real(u₀H_1D[2]), "u₂H_1D_im" => imag(u₀H_1D[2])])
    @printf("t=%.6f,norm L₂u₁=%.6f,norm L₂u₂=%.6f\n",t,norm_L2(u₀H_1D[1],dΩ_1D),norm_L2(u₀H_1D[2],dΩ_1D))
  end
end

t=0.000000,norm L₂u₁=0.000000,norm L₂u₂=1.000000
t=0.000001,norm L₂u₁=0.000000,norm L₂u₂=1.000000


t=0.000001,norm L₂u₁=0.000000,norm L₂u₂=1.000000
t=0.000002,norm L₂u₁=0.000000,norm L₂u₂=1.000000


t=0.000002,norm L₂u₁=0.000000,norm L₂u₂=1.000000
t=0.000003,norm L₂u₁=0.000000,norm L₂u₂=1.000000


t=0.000003,norm L₂u₁=0.000000,norm L₂u₂=1.000000
t=0.000004,norm L₂u₁=0.000000,norm L₂u₂=1.000000


t=0.000004,norm L₂u₁=0.000000,norm L₂u₂=1.000000
t=0.000005,norm L₂u₁=0.000000,norm L₂u₂=1.000000


t=0.000005,norm L₂u₁=0.000000,norm L₂u₂=1.000000
t=0.000006,norm L₂u₁=0.000000,norm L₂u₂=1.000000


t=0.000006,norm L₂u₁=0.000000,norm L₂u₂=1.000000
t=0.000007,norm L₂u₁=0.000000,norm L₂u₂=1.000000


t=0.000007,norm L₂u₁=0.000000,norm L₂u₂=1.000000
t=0.000008,norm L₂u₁=0.000000,norm L₂u₂=1.000000


t=0.000008,norm L₂u₁=0.000000,norm L₂u₂=1.000000
t=0.000009,norm L₂u₁=0.000000,norm L₂u₂=1.000000


t=0.000009,norm L₂u₁=0.000000,norm L₂u₂=1.000000
t=0.000010,norm L₂u₁=0.000000,norm L₂u₂=1.000000


### Mostramos salida de Densidad de probabilidad con visualizador Visit

<video controls width="500">
  <source src="./videos/pseudocolor_density1D_v3.mp4" type="video/mp4" />
</video>
